In [1]:
import numpy as np
import math
from __future__ import division

#Get Matrix A and A-Bar
class GenAllocationMatrix():
    def __init__(self,K,L):
        self.T=int(math.ceil(math.log(K+1,2))+L-1)
        self.MatrixABar=self.GenMatrixABar(K,L,self.T)
        
        
    def GenMatrixA(self,K,L,T):
        MatrixA=np.zeros((L,T+1),dtype='int')
        for col in range(0,T+1):
            for row in range(0,L):
                if col<row+1:
                    break;
                elif row==L-1 and col>L:
                    MatrixA[row][col]=min(1+2*MatrixA[row][col-1],K)
                else:
                    MatrixA[row][col]=min(2**(col-row-1),K)
        return MatrixA 
    
    
    def GenMatrixABar(self,K,L,T):
        MatrixA=self.GenMatrixA(K,L,T) 
        MatrixABar=np.zeros((L,T),dtype='int')
        for col in range(0,T):
            MatrixABar[:,col]=MatrixA[:,col+1]-MatrixA[:,col]
        return MatrixABar
           

In [2]:
#Generate Matrix Rx
class Rx():
    def __init__(self,K,L,MatrixABar):
        self.T=math.ceil(math.log(K+1,2))+L-1
        self.MatrixRx=self.GenRx(K,L,int(self.T),MatrixABar)
    
    def GenRx(self,K,L,T,MatrixABar):
        Rx=np.full((K, int(T)),-1)
        PhaseTwoFlag=int(math.ceil(math.log(K+1,2))-1)
        UntouchedFlag=0                              
        
        #1st phase, send symbols to all the untouched nodes
        for t in range(0,PhaseTwoFlag):                                                                 
                                                                          
            ChangeFlag=np.zeros((K,1),dtype = 'int') 
            for row in range(0,K):                   
                if Rx[row].sum()==-T:
                    UntouchedFlag=row
                    break;
            
            for l in range(0,L):  
                if MatrixABar[l][t]==0:
                    continue;
                S_l=l+1
                SymCount= MatrixABar[l][t]
                Count=0
                Current=0
                #Send the next symbol when the current one is finished. Send the oldest symbol first.
                while Count<SymCount:
                    if UntouchedFlag<K:      
                        Rx[UntouchedFlag][t]=S_l
                        ChangeFlag[UntouchedFlag]=1
                        UntouchedFlag=UntouchedFlag+1
                        Count=Count+1
        
        #2nd phase, 1)send symbols to the untouched nodes, 2)send S_1, 
        #3)send symbols other than s_1 to the available destination nodes
        #that have received the first symbol before.
        t=PhaseTwoFlag
        ChangeFlag=np.zeros((K,1),dtype = 'int')  #Save the changed rows at each time slot
        for l in range(0,L):  
            if MatrixABar[l][t]==0:
                continue;
            S_l=l+1
            SymCount= MatrixABar[l][t]
            Count=0
            Current=0
            while Count<SymCount:
                if UntouchedFlag!=0 and UntouchedFlag<K:      #Send to the receivers that haven't received any symbol first.
                    Rx[UntouchedFlag][t]=S_l
                    ChangeFlag[UntouchedFlag]=1
                    UntouchedFlag=UntouchedFlag+1
                    Count=Count+1
                    continue;
                if S_l !=1: 
                    if 1 in Rx[Current].tolist() and ChangeFlag[Current][0]==0:    
                        Rx[Current][t]=S_l
                        ChangeFlag[Current]=1
                        Count=Count+1
                        Current=Current+1
                        continue;
                    else:
                        Current=Current+1
                        continue;
                if S_l not in Rx[Current].tolist() and ChangeFlag[Current][0]==0:
                    Rx[Current][t]=S_l
                    ChangeFlag[Current]=1
                    Count=Count+1
                    Current=Current+1
                else:
                    Current=Current+1
                    
        #3rd Phase. Send symbols using greedy method.            
        for t in range((PhaseTwoFlag+1),T): 
            FailureCount=0
            ChangeFlag=np.zeros((K,1),dtype = 'int')  #Save the changed rows at each time slot
            
            for l in range(0,L):  
                if MatrixABar[l][t]==0:
                    continue;
                S_l=l+1
                SymCount= MatrixABar[l][t]
                Count=0
                Current=0
                #Send the next symbol when the current one is finished. 
                #Send the symbol sequentially from the first to the last.
                while Count<SymCount:
                    if S_l not in Rx[Current].tolist() and ChangeFlag[Current][0]==0:
                        Rx[Current][t]=S_l
                        ChangeFlag[Current]=1
                        Count=Count+1
                        Current=Current+1
                    else:
                        Current=Current+1
                        FailureCount+1
                    # Return False when cannot find available destination node.
                        if FailureCount>K:
                            return False                 
        return Rx

In [3]:
#Get Matrix Tx
class Tx():
    def __init__(self,K,L,Rx):
        self.SuccessFlag=1
        self.MatrixTx=self.GenTx(K,L,Rx)
                
    def GenTx(self,K,L,Rx):                            
        if isinstance(Rx, bool):
            return False           
        T=math.ceil(math.log(K+1,2))+L-1    
        Tx=np.full((K, int(T)),-1)
        for t in range(0,int(T)):
            ChangeFlag=np.zeros((K,1),dtype = 'int')
            SentFlag=np.zeros((K,1),dtype='int')
            
            ##A lastest symbol is sent by the source
            MaxRow=np.argmax(Rx,axis=0)[t]             
            Tx[MaxRow][t]=0                              
            ChangeFlag[MaxRow][0]=1
                        
            RowSort=np.argsort(Rx[:,t],kind='mergesort')    #Consider from the lastest to the oldest.
            FlagFinish=len(RowSort)-1
            FlagStart=np.argwhere(RowSort == MaxRow)[0][0]
            SaveFlagStart=FlagStart
            while FlagFinish>=0:
                while FlagStart<=FlagFinish:
                    j=RowSort[FlagStart]
                    if ChangeFlag[j][0]==1 or Rx[j][t]==-1:  
                        FlagStart=FlagStart+1
                        continue; 
                        
                    #Search the buffer of the nodes,from the first to the last.
                    for i in range (0,K):                     
                        ContinueFlag=0    
                        if  j==i or SentFlag[i][0]==1:         
                            ContinueFlag=1                                                       
                        if Rx[j][t] in Rx[i,0:t] and ContinueFlag!=1: 
                            Tx[j][t]=i+1
                            ChangeFlag[j][0]=1
                            SentFlag[i][0]=1
                            break;
                        elif i==K-1:                          #If no qualified transmitter return False
                            self.SuccessFlag=0
                            return False
                    FlagStart=FlagStart+1
                FlagFinish=SaveFlagStart-1
                FlagStart =np.argwhere(RowSort==np.argwhere(Rx[:,t]==Rx[RowSort[FlagFinish]][t])[0][0])[0][0]
                SaveFlagStart=FlagStart
                
        #Count=0                        #Double check，time wasting.
        #for i in range(0,K):
        #    for j in range(0,int(T)):
        #        if Tx[i][j]!=-1:
        #             Count=Count+1
        #if Count!=K*L:
        #    self.SuccessFlag=0
        #    return False       
        return Tx  

In [4]:
class Sample():         
    def __init__(self,K,L):
        self.ABar=GenAllocationMatrix(K,L).MatrixABar
        self.Rx=Rx(K,L,self.ABar).MatrixRx
        self.Tx=Tx(K,L,self.Rx).MatrixTx
        self.SuccessFlag=Tx(K,L,self.Rx).SuccessFlag
#        (self.SRx,self.STx)=self.ConvertToSender(K,L,self.Rx,self.Tx)
                                                  
#    def ConvertToSender(self,K,L,Rx,Tx):
#            T=int(math.ceil(math.log(K+1,2))+L-1)
#            SRx=np.full(((K+1), T),-1)
#            STx=np.full(((K+1), T),-1)
#            for t in range(0,T):
#                for k in range(0,K):
#                    s=Tx[k][t]
#                    if s==-1:
#                        continue;
#                    STx[s][t]=k+1
#                    SRx[s][t]=Rx[k][t]
#            return (SRx,STx)    

In [9]:
#Functions used to align the element of matrices
def Devide(y):
    y=map(Align,y)
    y=' '.join(y)
    return str(y)

def Align(x):
    if x==-1:
        x='*'
    x='{0: <3}'.format(x)
    return x

In [21]:
#Get the result for different cases.
f = open ('100-100 result.txt','w')
for K in range(1,101):
    for L in range (1,101):
        Test=Sample(K,L)
        print ("The case with K=",K,"L=",L,":", file = f ) 
        if isinstance(Test.Rx,bool):
            print ("FAIL", file = f ) 
        else:
            Rx2 = map(Devide,Test.Rx) 
            print ('The Rx is', file = f)
            for items in Rx2:
                item=''.join(items.split(","))
                print (item, file = f)
        
            Tx2 = map(Devide,Test.Tx)    
            print ('The Tx is', file = f)   
            for items in Tx2:
                item=''.join(items.split(","))
                print (item, file = f)
f.close()

#    SRx=map(Devide,Test.SRx)
#    print 'The SRx is'   
#    for items in SRx:
#        item=''.join(items.split(","))
#        print item

#    STx=map(Devide,Test.STx)    
#    print 'The STx is'   
#    for items in STx:
#        item=''.join(items.split(","))
#        print item